In [1]:
import torch
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models

In [2]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback
import socket
socket.setdefaulttimeout(30)
wandb.login()
wandb.init(project ='DL_assignment_2_B')

2024-04-07 08:35:58.150750: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 08:35:58.150869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 08:35:58.279925: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs23m066 (dlassignment). Use `wandb login --relogin` to force relogin


In [3]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip 

--2024-04-07 08:36:52--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.112.207, 74.125.124.207, 142.250.152.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.112.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: 'nature_12K.zip'

nature_12K.zip      100%[===================>]   3.55G   243MB/s    in 14s     

2024-04-07 08:37:05 (265 MB/s) - 'nature_12K.zip' saved [3816687935/3816687935]



In [4]:
!rm nature_12K.zip

In [5]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))])

train_dataset = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform)
train_dataset,val_dataset = torch.utils.data.random_split(train_dataset,[8000,1999])

transform2 = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(10),      # Randomly rotate the image by a maximum of 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
    transforms.RandomResizedCrop(224),  # Randomly crop and resize the image to 224x224
    transforms.ToTensor(),              # Convert the image to a PyTorch tensor
    transforms.Normalize((0.5,),(0.5,))  # Normalize the image
])

train_dataset2 = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform2)
train_dataset_aug,val_dataset_aug = torch.utils.data.random_split(train_dataset2,[8000,1999])

def dataFun(aug,batchSize):
    if(aug == 'no'):
        train_loader = torch.utils.data.DataLoader(train_dataset,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
        val_loader = torch.utils.data.DataLoader(val_dataset,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
        return train_loader,val_loader
    else:
        train_loader_aug = torch.utils.data.DataLoader(train_dataset_aug,batch_size =batchSize,shuffle = True,num_workers=4,pin_memory=True)
        val_loader_aug = torch.utils.data.DataLoader(val_dataset_aug,batch_size =batchSize,shuffle = True,num_workers=4,pin_memory=True)
        return train_loader_aug,val_loader_aug

In [6]:
def RESNET50(NUM_OF_CLASSES):
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, NUM_OF_CLASSES)
    
    for param in model.parameters():
        param.requires_grad = False
        
    for param in model.fc.parameters():
        param.requires_grad = True
   
    return model

In [7]:
def RESNET50_1(k,NUM_OF_CLASSES):
    model = models.resnet50(pretrained=True)    
    
    params = list(model.parameters())
    for param in params[:k]:
        param.requires_grad = False
        
    num_ftrs = model.fc.in_features
    
    model.fc = torch.nn.Linear(num_ftrs, NUM_OF_CLASSES)
    
    return model

In [9]:
def RESNET50_2(neurons_dense,NUM_OF_CLASSES):
    
    model = models.resnet50(pretrained=True)    
    
    activation_function_layer = nn.ReLU()
    
    for params in model.parameters():
        params.requires_grad = False
        
    num_ftrs = model.fc.in_features
    
    model.fc = nn.Sequential(
      nn.Linear(num_ftrs,neurons_dense),
      activation_function_layer,
      nn.Dropout(0.4),
      nn.Linear(neurons_dense, 10)
    )
    for param in model.fc.parameters():
        param.requires_grad = True
    
    return model

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
def train_fun(batchSize,num_epochs,learning_rate,aug,strategy,NUM_OF_CLASSES):
    aug = 'no'
    
    train_loader,val_loader = dataFun(aug,batchSize) 
    if(strategy == 0):
        model = RESNET50(NUM_OF_CLASSES).to(device)
    elif(strategy == 1):
        model = RESNET50_1(10,NUM_OF_CLASSES).to(device)
    else:
        model = RESNET50_2(256,NUM_OF_CLASSES).to(device)
        
        
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)
            # forward
            scores = model(data)
            loss = criterion(scores,targets)
            # backward
            optimizer.zero_grad()
            loss.backward()
            # gradient descent or adam step
            optimizer.step() #updates the parameters
        train_accuracy,train_loss = check_accuracy(train_loader, model,criterion,batchSize)
        validation_accuracy,validation_loss = check_accuracy(val_loader, model,criterion,batchSize)
        print(f"train_accuracy:{train_accuracy:.4f},train_loss:{train_loss:.4f}")
        print(f"validation_accuracy:{validation_accuracy:.4f},validation_loss:{validation_loss:.4f}")
        wandb.log({'train_accuracy':train_accuracy})
        wandb.log({'train_loss':train_loss})
        wandb.log({'val_accuracy':validation_accuracy})
        wandb.log({'val_loss':validation_loss})
        
    #wandb.log({'train_accuracy':train_accuracy})


In [13]:
def check_accuracy(loader,model,criterion,batchSize):
    num_correct = 0
    num_loss = 0
    total = 0
    num_samples = 0
    total_loss = 0.0 
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            scores = model(x)
            loss = criterion(scores, y)
            total_loss += loss.item()*batchSize
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum().item()
            num_samples += predictions.size(0)
    model.train()
    return (num_correct / num_samples)*100 , total_loss

In [ ]:
batchSize = 32
num_epochs = 30
learning_rate = 1e-4
aug = 'no'
NUM_OF_CLASSES = 10
strategy = 2
train_fun(batchSize,num_epochs,learning_rate,aug,strategy,NUM_OF_CLASSES)

In [14]:
def main_fun():
    wandb.init(project ='DL_assignment_2_B')
    params = wandb.config
    with wandb.init(project = 'DL_assignment_2_B', name='strategy'+str(params.strategy)+'batchSize'+str(params.batchSize)+'num_epochs'+str(params.num_epochs)+'learning_rate'+str(params.learning_rate)) as run:
        train_fun(params.batchSize,params.num_epochs,params.learning_rate,params.aug,params.strategy,10)
        
sweep_params = {
    'method' : 'bayes',
    'name'   : 'Shriram',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'val_accuracy',
    },
    'parameters' : {
        'batchSize':{'values':[32,64]},
        'num_epochs':{'values':[10,5]},
        'learning_rate':{'values' : [1e-3,1e-4]},
        'aug' :{'values':['no','yes']},
        'strategy' :{'values' : [2]}
    }
}
sweepId = wandb.sweep(sweep_params,project = 'DL_assignment_2_B')
wandb.agent(sweepId,function =main_fun,count = 2)
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: pq25pp5r
Sweep URL: https://wandb.ai/dlassignment/DL_assignment_2_B/sweeps/pq25pp5r


wandb: Agent Starting Run: na2eekxx with config:
wandb: 	aug: no
wandb: 	batchSize: 32
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 10
wandb: 	strategy: 2


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

100%|██████████| 125/125 [00:45<00:00,  2.77it/s]


train_accuracy:79.7250,train_loss:4796.9753
validation_accuracy:75.2876,validation_loss:1517.3449


train_accuracy,▁▄▆▇█
train_loss,█▅▃▂▁
val_accuracy,▁▄▇▇█
val_loss,█▅▂▂▁
train_accuracy,79.725
train_loss,4796.97534
val_accuracy,75.28764
val_loss,1517.34486


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7c7f283a9600>> (for post_run_cell), with arguments args (<ExecutionResult object at 7c7df0594d90, execution_count=14 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7c7df0596da0, raw_cell="def main_fun():
    wandb.init(project ='DL_assign.." store_history=True silent=False shell_futures=True cell_id=9426ab7b-71f6-4fe9-8bb5-aadaf8ab4872> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
import pandas as pd
data = {'LayerName': ['Image', 'conv1', 'pool1','conv2','pool2','conv3','pool3','conv4','pool4','conv5','pool5','DenseLayer','Output'],
        'Complete Tensor Size': ['256*256*3','((257-1k)^2)*m','((258-2k)^2)*m','((259-3k)^2)*m','((260-4k)^2)*m','((261-5k)^2)*m','((262-6k)^2)*m','((263-7k)^2)*m','((264-8k)^2)*m','((265-9k)^2)*m','((266-10k)^2)*m','((266-10k)^2)*m*n','n*10'],
        'Multiplications': ['0','3m(k^2)*(257-k)^2','0','(k^2*m^2)*(259-3k)^2','0','(k^2*m^2)*(261-5k)^2','0','(k^2*m^2)*(263-7k)^2','0','(k^2*m^2)*(265-9k)^2','0','(266-10k)^2*m*n','n*10'],
        'Additions': ['0','3m(k^2-1)*(257-k)^2','0','m^2*(k^2-1)*(259-3k)^2','0','(m^2)*(k^2-1)*(261-5k)^2','0','(m^2)*(k^2-1)*(263-7k)^2','0','(m^2)*(k^2-1)*(265-9k)^2','0','n*((266-10k)^2*m-1)','10*(n-1)'],
       }

df = pd.DataFrame(data)
df.head(13)

import matplotlib.pyplot as plt

# Plot the DataFrame as a table
plt.figure(figsize=(15,5))
ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table = plt.table(cellText=df.values, colLabels=df.columns, loc='center')
table.scale(1, 1.5)
table.auto_set_font_size(False)
table.set_fontsize(10)
plt.title('Computations')
plt.savefig('table4.png', bbox_inches='tight', pad_inches=0.05)
wandb.log({"Computations4": wandb.Image('table4.png')})
plt.show()
